In [57]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import pickle
import numpy as np
import random

In [50]:
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric, stem_text
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf

In [51]:
vote_file = pd.read_csv('brexit_full_updated3.csv')
vote_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN


In [52]:
politicians_party = {}
politicians = vote_file['full name'].unique()
for index, row in vote_file.iterrows():
    politicians_party[row['full name']] = row['party']

In [53]:
df1 = pd.read_csv('2015_commons.csv')
df2 = pd.read_csv('2016_commons.csv')
df = pd.concat([df1, df2])
df = df.drop(['Government'], axis=1)
df = df.loc[(df['Party'] == 'Conservative') | (df['Party'] == 'Labour')]
df = df.loc[df['Name'].isin(politicians)]
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] > '2015-5-1') & (df['Date'] <= '2016-6-30')
df = df.loc[mask]
df['party2'] = df['Name'].apply(lambda x: politicians_party[x])
# df = df.loc[df['Topic'] == 'European Union']
cols = df.columns
cols = ['date', 'speaker', 'name', 'Party', 'Topic', 'transcript', 'party2']
df.columns = cols
df.head()

,date,speaker,name,Party,Topic,transcript,party2
18469,2015-05-27,Mr Simon Burns (Chelmsford) (Con),Simon Burns,Conservative,admin,"I beg to move, That an humble Address be pres...",0.0
18470,2015-05-27,Mrs Sheryll Murray (South East Cornwall) (Con),Sheryll Murray,Conservative,admin,It is an honour to be invited to second this H...,1.0
18471,2015-05-27,Ms Harriet Harman (Camberwell and Peckham) (Lab),Harriet Harman,Labour,admin,I am sure the whole House will want to pay tri...,0.0
18472,2015-05-27,The Prime Minister (Mr David Cameron),David Cameron,Conservative,admin,"As we meet today, we should start, as the righ...",0.0
18473,2015-05-27,Mr David Winnick (Walsall North) (Lab),David Winnick,Labour,admin,The right hon. Gentleman is talking about “one...,0.0


In [78]:
X = []
Y = []

X_test = []

for index, row in df.iterrows():
    party = row['Party']
    party2 = row['party2']
    if party == 'Conservative' and party2 == 1:
        X.append(row['transcript'])
        Y.append(0)
    if party == 'Labour' and party2 == 0:
        X.append(row['transcript'])
        Y.append(1)
        
    if party == 'Conservative' and party2 == 0:
        X_test.append(row['transcript'])

In [79]:
c = list(zip(X, Y))
random.shuffle(c)
X, Y = zip(*c)

In [80]:
def strip_short2(text):
    return strip_short(text, minsize=4)

def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)


preprocessed_X = parmap.map(preprocess_text, X, pm_pbar=True)

30592it [00:01, 21236.23it/s]            


In [81]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_X)]

In [82]:
model = Doc2Vec(documents, vector_size=100, window=3, min_count=1, workers=4)

In [83]:
def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(1, activation='softmax')
    ])
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0, name='binary_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

In [84]:
def get_doc_vec(doc):
    return model.infer_vector(doc.words)

X_vec = parmap.map(get_doc_vec, documents, pm_pbar=True)

30592it [00:02, 11235.11it/s]                          


In [85]:
X_vec = np.array(X_vec)
Y = np.array(Y)
print(X_vec.shape)

(30551, 100)


In [86]:
classifier = build_network()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# class_weights = class_weight.compute_class_weight('balanced', np.unique(Y), Y)
classifier.fit(X_vec, Y, batch_size=64, epochs=25)

Epoch 1/25
478/478 [==============================] - 1s 1ms/step - loss: 2.2847 - accuracy: 0.5547
Epoch 2/25
478/478 [==============================] - 1s 1ms/step - loss: 0.6874 - accuracy: 0.5547
Epoch 3/25
478/478 [==============================] - 1s 2ms/step - loss: 0.6873 - accuracy: 0.5547
Epoch 4/25
478/478 [==============================] - 1s 1ms/step - loss: 0.6873 - accuracy: 0.5547
Epoch 5/25
478/478 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5547
Epoch 6/25
478/478 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5547
Epoch 7/25
478/478 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5547
Epoch 8/25
478/478 [==============================] - 1s 1ms/step - loss: 0.6873 - accuracy: 0.5547
Epoch 9/25
478/478 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5547
Epoch 10/25
478/478 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5547

In [ ]:
X_test = np.array(X_test)

In [88]:
preprocessed_X_test = parmap.map(preprocess_text, X_test, pm_pbar=True)
tagged_X_test = [models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_X_test)]

26496it [00:01, 19875.87it/s]            


In [89]:
inputs = parmap.map(get_doc_vec, tagged_X_test, pm_pbar=True)

26496it [00:02, 13162.04it/s]                           


In [90]:
inputs = np.array(inputs)

In [91]:
classifier.predict(inputs)

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)